### 通过Xpath从HTML当中提取所需要的值
* scrapy基于lxml提供了通过xpath或者css选择器来提取数证据的接口

<img scr='1.png' width=300px>

<img scr='1.png' width=300px>

<img src='1.png' width=500>

<img scr='1.png' width=300px>

<img src='2.png' width=500>
1. /article  (一般html源文件的根节点都是\<html\>,但是xml中可以自定义根元素)
2. /article/a  (这个a是子元素，不是后代元素)

<img src='3.png' width=500>

<img src='4.png' width=500>

#### 爬取http://blog.jobbole.com/110287/

<img src='5.png' width=500>

##### 获取标题，并分析元素路径
<img src='9.png' width=1000>

In [4]:
'''
获取标题
response.xpath返回一个SelectorList

class JobboleSpider(scrapy.Spider):
    name = 'jobbole'
    allowed_domains = ['blog.jobbole.com']
    start_urls = ['http://blog.jobbole.com/110287/']

    def parse(self, response):
        re_selector = response.xpath('//*[@id="post-110287"]/div[1]/h1')  # Chrome
        re2_selector = response.xpath('/html/body/div[3]/div[3]/div[1]/div[1]/h1')   # Firefox   
        pass
'''

'\n获取标题\nresponse.xpath返回一个SelectorList\n\nclass JobboleSpider(scrapy.Spider):\n    name = \'jobbole\'\n    allowed_domains = [\'blog.jobbole.com\']\n    start_urls = [\'http://blog.jobbole.com/110287/\']\n\n    def parse(self, response):\n        re_selector = response.xpath(\'//*[@id="post-110287"]/div[1]/h1\')  # Chrome\n        re2_selector = response.xpath(\'/html/body/div[3]/div[3]/div[1]/div[1]/h1\')   # Firefox   \n        pass\n'

<img src='6.png' width=1000>
<img src='7.png' width=1000>

可以发现下面的那行代码产生的SelectorList内容为空  
___
替换代码  
re2_selector = response.xpath('/html/body/div[1]/div[3]/div[1]/div[1]/h1')
<img src='8.png' width=1000>

1. 之所以把div\[3\]换成div\[1\]就能正确显示，是因为网页源代码里，h1所在位置是存在于第一个div里的
2. 之所以一开始分析的时候是div\[3\]是因为这些div元素是通过JS代码生成的，不是一开始就有的
3. 写入xpath路径里的元素必须是源文件里一开始就有的，不能是通过JS生成的

获取标题内容：通过text()  
response.xpath('//*[@id="post-110287"]/div[1]/h1/text()').extract()[0]  
___
另一种写法同样能选中标题  
title= response.xpath('//div[@class="entry-header"]/h1/text()').extract()[0] 
___
一般来讲,获取第一个元素有一个特殊的方法：extract_first("")  
因为往往爬取的元素都是没有内容的，也就是extract()[0]为NULL，因此需要额外的异常处理  
但是extract_first("")自动为我们实现了异常处理，即我们不需要额外的编写异常处理函数
___
获取时间：extract()为提取内容，strip()为消去左右两边空字符和换行符，replace()为替换字符  
create_time = response.xpath('//p[@class="entry-meta-hide-on-mobile"]/text()').extract()[0].strip().replace('·', '').strip()  
___  
<img src='10.png' width=700>  
获取点赞数：因为vote-post-up并不是唯一的一个class值，所以不能使用@class='...'的形式，只能使用函数的形式  
contains(选取属性,属性值)  
praise_num = int(response.xpath("//span[contains(@class, 'vote-post-up')]/h10/text()").extract()[0])  
___  
获取收藏数：  
response.xpath("//span[contains(@class, 'bookmark-btn')]/text()").extract()[0]  
Out[27]: ' 28 收藏'  
这个字符串里既有数字也有字符，如果要提取数字，就必须使用正则表达式  
```
import re
fav_num = response.xpath("//span[contains(@class, 'bookmark-btn')]/text()").extract()[0]
match_re = re.match('.*?(\d+).*', 'fav_num')
if match_re:
    fav_num = match_re.group(1)
```
___
获取评论数：
```
comments_num = response.xpath("//span[@href='#article-comment']/span/text()").extract()[0]
match_re = re.match('.*?(\d+).*', 'comments_num')
if match_re:
    comments_num = match_re.group(1)
```
___
获取tag：    
* tag_list = response.xpath('//p[@class="entry-meta-hide-on-mobile"]/a/text()').extract()  
tag_list = [element for element in tag_list if not element.strip().endswith('评论')]  
Out[18]: ['职场', '面试']  
* tags = ','.join(tag_list)  
Out[21]: '职场,面试'

Tips(在Shell命令行里调试scrapy代码，得先进入运行环境)：  
* scrapy shell 域名